In [2]:
# 載入必要模組
import pandas as pd
import mplfinance as mpf
import numpy as np
from talib.abstract import *
from haohaninfo import GOrder

In [3]:
# 將 K 線轉為 Dataframe
def KbarToDf(KBar):
    # 將 K 線 Dictionary 轉換成 Dataframe
    Kbar_df = pd.DataFrame(KBar)
    # 將 Dataframe 欄位名稱轉換
    Kbar_df.columns = [i[0].upper()+i[1:]for i in Kbar_df.columns]
    # 將 Time 欄位設為索引
    Kbar_df.set_index("Time",inplace=True)
    # 回傳
    return Kbar_df

In [4]:
# 繪製 K 線圖
def ChartKBar(KBar,addp=[],volume_enable=True):
    # 將 K 線轉為 Dataframe
    Kbar_df = KbarToDf(KBar)
    # 開始繪圖
    mpf.plot(Kbar_df,volume=volume_enable,addplot=addp,type='candle',style='charles')


In [5]:
# 繪製 K 線圖以及 MA
def ChartKBar_MA(KBar,longPeriod=20,shortPeriod=5):
    # 計算移動平均線(長短線)
    KBar['MA_long'] = SMA(KBar, timeperiod=longPeriod)
    KBar['MA_short'] = SMA(KBar, timeperiod=shortPeriod)
    # 將 K 線轉為 Dataframe
    Kbar_df = KbarToDf(KBar)
    # 將副圖繪製出來
    addp = []
    addp.append(mpf.make_addplot(Kbar_df['MA_long'],color='red'))
    addp.append(mpf.make_addplot(Kbar_df['MA_short'],color='yellow'))
    # 開始繪圖
    ChartKBar(KBar,addp,True)

In [6]:
# 繪製 K 線圖以及 RSI
def ChartKBar_RSI(KBar,RSIPeriod,upper,lower):
    # 計算 RSI 以及買超賣超線
    KBar['RSI'] = RSI(KBar,timeperiod = RSIPeriod)
    KBar['Ceil'] = np.array([upper]*len(KBar['time']))
    KBar['Floor'] = np.array([lower]*len(KBar['time']))
    # 將 K 線轉為 Dataframe
    Kbar_df = KbarToDf(KBar)
    
    # 將副圖繪製出來
    addp = []
    addp.append(mpf.make_addplot(Kbar_df['RSI'], panel='lower',color='black',secondary_y=False))
    addp.append(mpf.make_addplot(Kbar_df['Ceil'], panel='lower',color='red',secondary_y=False))
    addp.append(mpf.make_addplot(Kbar_df['Floor'], panel='lower',color='red',secondary_y=False))
    
    # 開始繪圖
    ChartKBar(KBar,addp,False)

In [7]:
KBar = GOrder.GetTAKBar('20200310','20200315','2303','Stock','0','5')
#ChartKBar_RSI(KBar,50,80,20)

generating data...
請先登入或開啟GOrder


In [8]:
# 繪製 K 線圖以及布林通道
def ChartKBar_BBANDS(KBar,BBANDSPeriod):
    
    # 計算布林通道
    KBar['Upper'],KBar['Middle'],KBar['Lower'] = BBANDS(KBar,timeperiod=BBANDSPeriod)
    
    # 將 K 線轉為 Dataframe
    Kbar_df = KbarToDf(KBar)
    
    # 將副圖繪製出來
    addp = []
    addp.append(mpf.make_addplot(Kbar_df['Upper'],color='yellow'))
    addp.append(mpf.make_addplot(Kbar_df['Middle'],color='grey'))
    addp.append(mpf.make_addplot(Kbar_df['Lower'],color='yellow'))
    
    # 開始繪圖
    ChartKBar(KBar,addp,True)

In [9]:
# 繪製 K 線圖加上 MA 以及下單紀錄
def ChartOrder_MA(KBar,TR):
    
    # 將 K 線轉為 Dataframe
    Kbar_df = KbarToDf(KBar)
    
    # 定義指標副圖
    addp = []
    addp.append(mpf.make_addplot(Kbar_df['long'],color='red'))
    addp.append(mpf.make_addplot(Kbar_df['short'],color='yellow'))
    
    # 繪製指標、下單圖
    ChartOrder(KBar,TR,addp)